# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
import re

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mohammad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [79]:
# load data from database
engine = create_engine('sqlite:///ETL_final.db')
df = pd.read_sql_table('cleaned_table', engine)
                      
X = df['message']
y = df.iloc[:,4:]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wnl = WordNetLemmatizer()

def tokenize(text):
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    cleaned_text = word_tokenize(cleaned_text)
    cleaned_text = [w for w in cleaned_text if w not in stopwords.words("english")]
    return [wnl.lemmatize(x) for x in cleaned_text]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
pipeline = Pipeline([
    ('tfidf_vectorizer', TfidfVectorizer(tokenizer=tokenize)),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y)

model = pipeline.fit(X_train, y_train)

In [12]:
y_preds = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [95]:
from sklearn.metrics import f1_score, precision_score, classification_report

y_test_numpy =y_test.to_numpy()
for i, column_name in zip(range(36), y_test.columns):
    print(f"column_name: {column_name} {classification_report(y_test_numpy[:,i], y_preds[:,i])}")
    

column_name: related               precision    recall  f1-score   support

           0       0.64      0.20      0.30      1532
           1       0.79      0.97      0.87      4979
           2       0.50      0.12      0.19        43

   micro avg       0.78      0.78      0.78      6554
   macro avg       0.64      0.43      0.45      6554
weighted avg       0.75      0.78      0.73      6554

column_name: request               precision    recall  f1-score   support

           0       0.91      0.96      0.94      5453
           1       0.73      0.54      0.62      1101

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.82      0.75      0.78      6554
weighted avg       0.88      0.89      0.88      6554

column_name: offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

   micro avg       0.99      0.99      0.99      6554
   macro avg 

column_name: hospitals               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6490
           1       0.25      0.09      0.14        64

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.62      0.55      0.57      6554
weighted avg       0.98      0.99      0.99      6554

column_name: shops               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

column_name: aid_centers               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.40      0.11      0.17        75

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.69      0.55      0.58      6554
weighte


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.